

## **Chapter 14: Testing GraphQL**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Design a comprehensive testing strategy for GraphQL APIs (Unit, Integration, E2E)
- Write isolated unit tests for individual resolvers using Jest
- Implement integration tests for the complete GraphQL schema using `apollo-server-testing`
- Mock DataSources and external APIs for reliable, fast tests
- Set up end-to-end tests that verify the full request/response cycle
- Implement contract testing to ensure schema compatibility between services
- Use snapshot testing for GraphQL responses
- Achieve high test coverage while maintaining test performance

---

## **Prerequisites**

- Completed Chapter 7: Building a GraphQL Server
- Completed Chapter 13: Monitoring and Observability
- Familiarity with Jest or similar JavaScript testing frameworks
- Understanding of mocking and stubbing concepts
- Basic knowledge of CI/CD pipelines (optional)

---

## **14.1 Testing Strategy: Unit vs. Integration vs. E2E**

A robust GraphQL testing strategy requires multiple layers. Unlike REST APIs where you test endpoints, GraphQL requires testing the schema, resolvers, and business logic independently and collectively.

**The Testing Pyramid for GraphQL:**

```
        /\
       /  \  E2E Tests (Few, slow, expensive)
      /____\     - Full HTTP requests
     /      \    - Database interactions
    /        \   
   /__________\ Integration Tests (Medium, moderate speed)
  /            \   - Schema-level testing
 /              \  - Resolver composition
/________________\ 
Unit Tests (Many, fast, cheap)
    - Individual resolvers
    - Utility functions
    - DataSource methods
```

**When to use each:**

| Test Type | Scope | Speed | Use Case |
|-----------|-------|-------|----------|
| **Unit** | Single resolver/function | Milliseconds | Business logic, edge cases, error handling |
| **Integration** | Schema + Resolvers | Hundreds of ms | Query execution, type checking, auth flows |
| **E2E** | Full HTTP stack | Seconds | Critical user journeys, deployment verification |

---

## **14.2 Unit Testing Resolvers**

Unit tests focus on isolated resolver logic without spinning up the full Apollo Server or hitting real databases.

### **Setup and Configuration**

**Install dependencies:**

```bash
npm install --save-dev jest @types/jest
```

**Jest configuration (`jest.config.js`):**

```javascript
module.exports = {
  testEnvironment: 'node',
  collectCoverageFrom: [
    'src/**/*.js',
    '!src/index.js', // Exclude entry point
  ],
  coverageThreshold: {
    global: {
      branches: 80,
      functions: 80,
      lines: 80,
      statements: 80
    }
  },
  testMatch: ['**/*.test.js'],
  verbose: true
};
```

### **Testing Pure Resolvers**

Simple resolvers that transform data are the easiest to test:

```javascript
// resolvers/user.js
const resolvers = {
  User: {
    // Pure function - easy to test
    fullName: (parent) => {
      return `${parent.firstName} ${parent.lastName}`;
    },
    
    // Resolver with conditional logic
    isAdult: (parent) => {
      return parent.age >= 18;
    },
    
    // Resolver with calculation
    accountAge: (parent) => {
      if (!parent.createdAt) return null;
      const diff = Date.now() - new Date(parent.createdAt).getTime();
      return Math.floor(diff / (1000 * 60 * 60 * 24)); // Days
    }
  }
};

module.exports = resolvers;
```

**Unit tests (`user.resolver.test.js`):**

```javascript
const resolvers = require('./user');

describe('User Resolvers', () => {
  describe('fullName', () => {
    it('should concatenate first and last name', () => {
      // Arrange
      const parent = { firstName: 'John', lastName: 'Doe' };
      
      // Act
      const result = resolvers.User.fullName(parent);
      
      // Assert
      expect(result).toBe('John Doe');
    });
    
    it('should handle missing last name', () => {
      const parent = { firstName: 'John', lastName: null };
      
      const result = resolvers.User.fullName(parent);
      
      expect(result).toBe('John null');
    });
  });
  
  describe('isAdult', () => {
    it('should return true for users 18 or older', () => {
      expect(resolvers.User.isAdult({ age: 18 })).toBe(true);
      expect(resolvers.User.isAdult({ age: 25 })).toBe(true);
    });
    
    it('should return false for users under 18', () => {
      expect(resolvers.User.isAdult({ age: 17 })).toBe(false);
      expect(resolvers.User.isAdult({ age: 0 })).toBe(false);
    });
  });
  
  describe('accountAge', () => {
    it('should calculate days since creation', () => {
      const tenDaysAgo = new Date(Date.now() - 10 * 24 * 60 * 60 * 1000);
      const parent = { createdAt: tenDaysAgo.toISOString() };
      
      const result = resolvers.User.accountAge(parent);
      
      expect(result).toBe(10);
    });
    
    it('should return null if no createdAt', () => {
      const result = resolvers.User.accountAge({ createdAt: null });
      
      expect(result).toBeNull();
    });
  });
});
```

### **Testing Resolvers with Dependencies**

When resolvers use DataSources or context, use dependency injection:

```javascript
// resolvers/query.js
const resolvers = {
  Query: {
    user: async (_, { id }, { dataSources, user }) => {
      // Authorization check
      if (!user) {
        throw new Error('Unauthorized');
      }
      
      // Fetch from DataSource
      const userData = await dataSources.userAPI.getUserById(id);
      
      if (!userData) {
        throw new Error('User not found');
      }
      
      return userData;
    }
  }
};
```

**Testing with mocked dependencies:**

```javascript
describe('Query.user resolver', () => {
  let mockDataSources;
  let mockContext;
  
  beforeEach(() => {
    mockDataSources = {
      userAPI: {
        getUserById: jest.fn()
      }
    };
    
    mockContext = {
      dataSources: mockDataSources,
      user: { id: '1', role: 'ADMIN' } // Authenticated user
    };
  });
  
  it('should return user when found', async () => {
    // Arrange
    const expectedUser = { id: '123', name: 'Test User' };
    mockDataSources.userAPI.getUserById.mockResolvedValue(expectedUser);
    
    // Act
    const result = await resolvers.Query.user(
      null,           // parent
      { id: '123' },  // args
      mockContext,    // context
      null            // info (usually not needed in unit tests)
    );
    
    // Assert
    expect(result).toEqual(expectedUser);
    expect(mockDataSources.userAPI.getUserById).toHaveBeenCalledWith('123');
    expect(mockDataSources.userAPI.getUserById).toHaveBeenCalledTimes(1);
  });
  
  it('should throw error when user not found', async () => {
    mockDataSources.userAPI.getUserById.mockResolvedValue(null);
    
    await expect(
      resolvers.Query.user(null, { id: '999' }, mockContext, null)
    ).rejects.toThrow('User not found');
  });
  
  it('should throw error when unauthenticated', async () => {
    const unauthenticatedContext = {
      ...mockContext,
      user: null
    };
    
    await expect(
      resolvers.Query.user(null, { id: '123' }, unauthenticatedContext, null)
    ).rejects.toThrow('Unauthorized');
  });
});
```

---

## **14.3 Integration Testing the Schema**

Integration tests verify that resolvers work together correctly with the type system. We use `apollo-server-testing` to create a test client.

### **Setup**

```bash
npm install --save-dev apollo-server-testing
```

**Test Utilities (`test-utils.js`):**

```javascript
const { ApolloServer } = require('apollo-server');
const { createTestClient } = require('apollo-server-testing');
const typeDefs = require('../src/schema');
const resolvers = require('../src/resolvers');

// Factory function to create test server with configurable context
const createTestServer = (context = {}) => {
  const server = new ApolloServer({
    typeDefs,
    resolvers,
    context: () => context,
    // Disable plugins that might interfere with tests
    plugins: [],
  });
  
  return createTestClient(server);
};

module.exports = { createTestServer };
```

### **Writing Integration Tests**

```javascript
const { gql } = require('apollo-server');
const { createTestServer } = require('./test-utils');

describe('User Queries Integration', () => {
  const GET_USER = gql`
    query GetUser($id: ID!) {
      user(id: $id) {
        id
        name
        email
      }
    }
  `;
  
  it('should fetch user by ID', async () => {
    // Arrange
    const { query } = createTestServer({
      user: { id: '1', role: 'USER' }, // Mock authenticated context
      dataSources: {
        userAPI: {
          getUserById: jest.fn().mockResolvedValue({
            id: '123',
            name: 'Test User',
            email: 'test@example.com'
          })
        }
      }
    });
    
    // Act
    const res = await query({
      query: GET_USER,
      variables: { id: '123' }
    });
    
    // Assert
    expect(res.errors).toBeUndefined();
    expect(res.data.user).toEqual({
      id: '123',
      name: 'Test User',
      email: 'test@example.com'
    });
  });
  
  it('should return error for invalid ID format', async () => {
    const { query } = createTestServer({
      user: { id: '1', role: 'USER' },
      dataSources: {
        userAPI: {
          getUserById: jest.fn().mockRejectedValue(new Error('Invalid ID'))
        }
      }
    });
    
    const res = await query({
      query: GET_USER,
      variables: { id: 'invalid' }
    });
    
    expect(res.errors).toBeDefined();
    expect(res.errors[0].message).toContain('Invalid ID');
  });
});
```

### **Testing Mutations**

```javascript
describe('User Mutations Integration', () => {
  const CREATE_USER = gql`
    mutation CreateUser($input: CreateUserInput!) {
      createUser(input: $input) {
        id
        name
        email
      }
    }
  `;
  
  it('should create user successfully', async () => {
    const mockUser = {
      id: 'new-id',
      name: 'New User',
      email: 'new@example.com'
    };
    
    const { mutate } = createTestServer({
      user: { id: '1', role: 'ADMIN' },
      dataSources: {
        userAPI: {
          createUser: jest.fn().mockResolvedValue(mockUser)
        }
      }
    });
    
    const res = await mutate({
      mutation: CREATE_USER,
      variables: {
        input: {
          name: 'New User',
          email: 'new@example.com'
        }
      }
    });
    
    expect(res.errors).toBeUndefined();
    expect(res.data.createUser).toEqual(mockUser);
  });
  
  it('should enforce authorization', async () => {
    const { mutate } = createTestServer({
      user: { id: '1', role: 'USER' }, // Non-admin user
      dataSources: {
        userAPI: {
          createUser: jest.fn()
        }
      }
    });
    
    const res = await mutate({
      mutation: CREATE_USER,
      variables: {
        input: { name: 'Test', email: 'test@test.com' }
      }
    });
    
    expect(res.errors).toBeDefined();
    expect(res.errors[0].message).toContain('Forbidden');
  });
});
```

---

## **14.4 Mocking GraphQL for Frontend Testing**

When testing React components that use Apollo Client, mock the GraphQL layer.

**Setup (`MockedProvider`):**

```javascript
import { MockedProvider } from '@apollo/client/testing';
import { render, screen, waitFor } from '@testing-library/react';
import UserProfile from './UserProfile';
import { GET_USER } from './queries';

const mocks = [
  {
    request: {
      query: GET_USER,
      variables: { id: '1' }
    },
    result: {
      data: {
        user: {
          id: '1',
          name: 'Test User',
          email: 'test@example.com'
        }
      }
    }
  },
  {
    request: {
      query: GET_USER,
      variables: { id: '2' }
    },
    error: new Error('User not found')
  }
];

describe('UserProfile Component', () => {
  it('should render user data', async () => {
    render(
      <MockedProvider mocks={mocks} addTypename={false}>
        <UserProfile userId="1" />
      </MockedProvider>
    );
    
    expect(screen.getByText(/loading/i)).toBeInTheDocument();
    
    await waitFor(() => {
      expect(screen.getByText('Test User')).toBeInTheDocument();
    });
  });
  
  it('should handle errors', async () => {
    render(
      <MockedProvider mocks={mocks} addTypename={false}>
        <UserProfile userId="2" />
      </MockedProvider>
    );
    
    await waitFor(() => {
      expect(screen.getByText(/error/i)).toBeInTheDocument();
    });
  });
});
```

---

## **14.5 Contract Testing**

Contract testing ensures that your GraphQL schema doesn't break clients. Use tools like **GraphQL Inspector** or **Apollo Rover** to detect breaking changes.

**Schema Diffing with Apollo Rover:**

```bash
# Compare local schema against production
npx @apollo/rover graph check my-graph@current \
  --schema ./schema.graphql
```

**CI/CD Integration (`.github/workflows/schema-check.yml`):**

```yaml
name: Schema Check
on: [pull_request]

jobs:
  check-schema:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v2
      
      - name: Install Rover
        run: npm install -g @apollo/rover
      
      - name: Check Schema
        run: |
          rover graph check my-graph@production \
            --schema ./src/schema.graphql
```

**Breaking Changes to Detect:**
- Removing a field
- Changing a field's type
- Making a non-nullable field nullable (safe) vs nullable to non-nullable (breaking)
- Removing enum values
- Changing argument types

---

## **Chapter Summary**

Comprehensive testing ensures your GraphQL API remains reliable as it evolves. Each testing layer serves a distinct purpose in maintaining quality.

### **Key Takeaways:**

1.  **Unit Tests**: Test individual resolver functions in isolation. Fast, focused on business logic and edge cases. Mock all dependencies (DataSources, context).
2.  **Integration Tests**: Test the complete schema execution using `apollo-server-testing`. Verify that types, resolvers, and middleware work together correctly.
3.  **E2E Tests**: Full HTTP tests for critical paths. Slow but essential for deployment confidence.
4.  **Mocking**: Use Jest mocks for DataSources and external APIs. Use `MockedProvider` for frontend component testing.
5.  **Contract Testing**: Prevent breaking changes in CI/CD. Use Apollo Rover or GraphQL Inspector to validate schema evolution.
6.  **Coverage**: Aim for 80%+ coverage, but focus on critical paths over arbitrary metrics.
7.  **Test Data**: Use factories or fixtures to generate test data consistently.

### **Testing Best Practices:**

- [ ] Unit test all complex resolver logic
- [ ] Integration test all queries and mutations
- [ ] Mock external APIs and databases
- [ ] Test error paths and edge cases (nulls, empty arrays)
- [ ] Use snapshot testing sparingly for response shapes
- [ ] Implement contract testing in CI/CD
- [ ] Keep tests fast (unit < 100ms, integration < 1s)
- [ ] Test authorization rules explicitly
- [ ] Use separate test database for integration tests
- [ ] Clean up resources after tests (close connections)

---

### **🚀 Next Up: Chapter 15 - Federation and Microservices**

**Summary:** We have built, secured, monitored, and tested a monolithic GraphQL API. But how do we scale to enterprise level? In Chapter 15, we will explore **Apollo Federation**, the industry standard for building distributed GraphQL architectures. You will learn how to split your monolith into microservices, implement entity references across services, and manage a unified data graph at scale. This is the final step toward production-grade GraphQL mastery.**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='13. monitoring_and_observability.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../6. advanced_architecture_and_scale/15. federation_and_microservices.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
